In [1]:
import numpy as np
import pandas as pd
import os
import time
import random
import zipfile
import shutil
from tqdm import tqdm
from scipy.integrate import solve_ivp

# Set up constants
NUM_DATASETS = 10000  # Number of parameter combinations
ITERATIONS = 10000    # Number of states for each trajectory
DT = 0.001            # Time step

# Parameter ranges
RHO_RANGE = (20, 100)
SIGMA_RANGE = (5, 45)
BETA_RANGE = (0.1, 5)

# Initial conditions range
IC_RANGE = (-30, 30)

def lorenz_system(t, state, sigma, rho, beta):
    """Compute the derivatives for the Lorenz system"""
    x, y, z = state
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return [dx, dy, dz]

def generate_trajectory(x0, y0, z0, sigma, rho, beta, iterations=10000):
    """Generate a complete trajectory for the Lorenz system using SciPy's RK45"""
    initial_state = [x0, y0, z0]
    t_span = (0, DT * iterations)
    t_eval = np.linspace(0, DT * iterations, iterations)

    solution = solve_ivp(
        fun=lambda t, y: lorenz_system(t, y, sigma, rho, beta),
        t_span=t_span,
        y0=initial_state,
        method='RK45',
        t_eval=t_eval,
        rtol=1e-8,
        atol=1e-8
    )

    if not solution.success:
        return None

    # Check for divergence or convergence
    max_values = np.max(np.abs(solution.y), axis=1)
    if np.any(max_values > 1e8) or np.all(np.abs(solution.y[:, -1]) < 1e-8):
        return None

    return solution.y.T  

In [2]:
def main():
    data_dir = "lorenz_data_separate"
    os.makedirs(data_dir, exist_ok=True)

    print(f"Starting generation of {NUM_DATASETS} Lorenz system datasets...")

    count = 0
    attempts = 0

    with tqdm(total=NUM_DATASETS) as pbar:
        while count < NUM_DATASETS and attempts < NUM_DATASETS * 2:
            attempts += 1

            sigma = random.uniform(*SIGMA_RANGE)
            rho = random.uniform(*RHO_RANGE)
            beta = random.uniform(*BETA_RANGE)

            # Random initial conditions
            x0 = random.uniform(*IC_RANGE)
            y0 = random.uniform(*IC_RANGE)
            z0 = random.uniform(*IC_RANGE)

            if abs(x0) < 1e-10 and abs(y0) < 1e-10 and abs(z0) < 1e-10:
                x0 = 0.1  # Avoid all-zero start

            data = generate_trajectory(x0, y0, z0, sigma, rho, beta, ITERATIONS)

            if data is None:
                continue

            count += 1
            pbar.update(1)

            # Create a unique filename based on parameters
            filename = f"{count + 10000}_{x0:.4f}_{y0:.4f}_{z0:.4f}_{sigma:.4f}_{rho:.4f}_{beta:.4f}.csv"            
            filepath = os.path.join(data_dir, filename)

            df = pd.DataFrame(data, columns=['x', 'y', 'z'])
            df.to_csv(filepath, index=False, float_format='%.5f')
    print(f"Successfully generated {count} datasets after {attempts} attempts!")
    print("Process complete!")

In [3]:
main()

Starting generation of 10000 Lorenz system datasets...


100%|██████████| 10000/10000 [52:30<00:00,  3.17it/s]

Successfully generated 10000 datasets after 10000 attempts!
Process complete!
